# Abstract Formulation 

**Data Variables:** 
- $I$: the set of nurses. 
- $J$: the set of shifts.
- $p_{ij}$: the preference score of nurse $i$ for shift $j$.
- $d_j$: the demand of nurses for shift $j$.
- $n$: the number of weeks to optimize schedule for.
- $W$: the set of shifts included in each week 0 through n-1, ($0-20$ for $w_0$, $21-41$ for $w_1$ and so on).
- $M$: maximum number of shifts a given nurse can work in a week.

**Decision Variables:** <br>
<br>
For each nurse $i \in I$ and each shift $j \in J$, let $x_{ij}$ denote whether nurse $i$ will work shift $j$. (Binary)

**Auxilary Variables:**
- $z_j$: variable indicating whether shift $j$ is a night shift or not. (Binary)
- $t_i$: the total number of shifts assigned to nurse $i$. (Integer)
- $tn_i$: the total number of night shifts assigned to nurse $i$. (Integer)
- $U$: the maximum number of shifts worked across all nurses. (Integer)
- $L$: the minimum number of shifts worked across all nurses. (Integer)
- $U_N$: the maximum number of night shifts worked across all nurses. (Integer)
- $L_N$: the minimum number of night shifts worked across all nurses. (Integer)

**Objective and constraints:**

$$\begin{aligned}
\text{Maximize:} && \sum_{i \in I, j\in J} x_{ij}p_{ij} - 100(U-L) - 150(U_N-L_N) \\
\text{subject to:} \\
\text{(Shift Requirements)} && \sum_{i \in I} x_{ij} & \ge d_j && \text{ for $j \in J$.}\\
\text{(Weekly Shifts Limit)} && \sum_{j \in w} x_{ij} & \le M && \text{ for $i \in I$, $w \in W$.}\\
\text{(No consecutive shifts)} && \sum_{j \in \text{{$1$,len($J$)}}} x_{ij} + x_{ij-1} & \le 1 && \text{ for $i \in I$}\\
&& \sum_{j \in \text{{$0$,len($J$)$-1$}}} x_{ij} + x_{ij+1} & \le 1 && \text{ for $i \in I$}\\
\text{(Night shift breaks)} && x_{ij-2} &\le 1-(z_j)(x_{ij}) && \text{ for $i \in I, j \in$ {2,len(J)}}\\
&& x_{ij-1} &\le 1-(z_j)(x_{ij}) && \text{ for $i \in I, j \in$ {1,len(J)}}\\
&& x_{ij+1} &\le 1-(z_j)(x_{ij}) && \text{ for $i \in I, j \in$ {0,len(J-1)}}\\
&& x_{ij+2} &\le 1-(z_j)(x_{ij}) && \text{ for $i \in I, j \in$ {0,len(J-2)}}\\
\text{(Blackout shifts)} && x_{ij} &\le p_{ij} && \text{ for $i \in I, j \in J$}\\
\text{(Total shifts)} && \sum_{j\in J}x_{ij} & = t_i && \text{ for $i \in I$}\\
\text{(Total night shifts)} && \sum_{j\in J}x_{ij}z_j & = tn_i && \text{ for $i \in I$}\\
\text{(Max Total shifts)} && U & = max(t_i)  && \text{ for $i \in I$}\\
\text{(Min Total shifts)} && L & = min(t_i)  && \text{ for $i \in I$}\\
\text{(Max Total night shifts)} && U_N & = max(tn_i)  && \text{ for $i \in I$}\\
\text{(Min Total night shifts)} && L_N & = min(tn_i)  && \text{ for $i \in I$}\\
\text{(Non-Negativity)} && x_{ij} & \ge 0 && \text{for $i \in I, j \in J$} \\
\end{aligned}$$

# Small Data

In [17]:
import pandas as pd
from gurobipy import Model, GRB, max_, min_

In [18]:
prefs=pd.read_excel('small_data.xlsx',header=[0,1,2],sheet_name='Preferences',index_col=0)
reqs =pd.read_excel('small_data.xlsx',sheet_name='Requirements',index_col=0)

In [19]:
names=prefs.index
shifts=prefs.columns
shift_id=shifts.get_level_values(2)
shift_id

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19, 20],
           dtype='int64', name='shift_id')

In [20]:
# Remove the MultiIndex
prefs.columns=shift_id
prefs

shift_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
name,,,,,,,,,,,,,,,,,,,,,
Alexis,1,1,1,1,2,1,1,0,2,1,1,2,2,2,1,2,2,2,1,1,2
Alyssa,2,2,0,1,2,1,2,2,2,1,2,2,2,1,2,1,2,2,0,0,0
Anthony,1,1,2,1,1,2,2,1,0,2,1,1,2,2,2,1,2,2,1,1,2
Brandon,1,1,2,2,2,1,1,2,1,0,0,0,2,1,0,0,1,2,2,2,1
Brianna,0,2,1,1,1,2,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0
Caleb,1,2,1,1,2,2,1,2,2,1,2,2,1,2,2,2,1,1,1,1,1
Cameron,2,2,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,2,2,2,1
Chloe,2,0,0,0,0,1,2,1,2,2,2,1,0,0,0,1,2,1,1,0,1
Christopher,1,1,1,1,1,1,2,1,2,2,2,2,1,2,2,2,1,1,1,1,2


In [21]:
I = names           #set of nurses
J = shift_id        #set of shifts
n = int(len(J)/21)  #number of weeks
W = range(n)        #set of weeks
d = reqs['persons'] #required # of nurses for each shift
nights = J[2::3]    #shift id of every night shift
z = []              #binary auxillary variable for whether or not it is a night shift
for j in J:
    if j in nights:
        z.append(1)
    else:
        z.append(0)
#prefs.loc[i,j]     #nurse i's preference for shift j

In [22]:
mod = Model()

x = mod.addVars(I,J,vtype=GRB.BINARY)        #DV: whether nurse i works shift j, binary
total = mod.addVars(I,vtype=GRB.INTEGER)     #total # of shifts worked for each nurse
totaln=mod.addVars(I,vtype=GRB.INTEGER)      #total # of night shift worked for each nurse
U = mod.addVar(vtype=GRB.INTEGER)            #max # of shifts worked across all nurses
L = mod.addVar(vtype=GRB.INTEGER)            #min # of shifts worked across all nurses
UN = mod.addVar(vtype=GRB.INTEGER)           #max # of night shifts worked across all nurses
LN = mod.addVar(vtype=GRB.INTEGER)           #min # of night shifts worked across all nurses

mod.setObjective(sum(x[i,j]*prefs.loc[i,j] for i in I for j in J) - 100*(U-L) - 150*(UN-LN), sense=GRB.MAXIMIZE)

mod.addConstr(U==max_([total[i] for i in I]))   #max total shifts
mod.addConstr(L==min_([total[i] for i in I]))   #min total shifts
mod.addConstr(UN==max_([totaln[i] for i in I])) #max total night shifts
mod.addConstr(LN==min_([totaln[i] for i in I])) #min total night shifts

for i in I:                                     #total number of shifts and nightshifts scheduled for each nurse
    mod.addConstr(total[i]==sum(x[i,j] for j in J))
for i in I:
    mod.addConstr(totaln[i]==sum(x[i,j]*z[j]for j in J))

for j in J:                                     #shift demand
    mod.addConstr(sum(x[i,j] for i in I)>=d[j])
    
for i in I:                                     #less than 6 shifts a week
    mod.addConstr(sum(x[i,j] for j in J)<=6)
    
for i in I:                                     #no consecutive shifts
    for j in J[1:]:
        mod.addConstr(x[i,j-1]+x[i,j]<=1)
for i in I:
    for j in J[:-1]:
        mod.addConstr(x[i,j+1]+x[i,j]<=1)
        
for i in I:                                     #night shift constraints
    for j in J[2:]:
        mod.addConstr(x[i,j-2]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[1:]:
        mod.addConstr(x[i,j-1]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[:-1]:
        mod.addConstr(x[i,j+1]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[:-2]:
        mod.addConstr(x[i,j+2]<=1-(z[j]*x[i,j]))

for i in I:                                     #blackout shift
    for j in J:
        mod.addConstr(x[i,j]<=prefs.loc[i,j])

mod.setParam('outputflag',False)
mod.optimize()
mod.objval
prefsum = (sum(x[i,j].x*prefs.loc[i,j] for i in I for j in J)) #sum of preference score
print(mod.objval)
print(prefsum)
print(U.x-L.x)   #shift inequality
print(UN.x-LN.x) #night shift inequality

-667.0
83.0
3.0
3.0


In [23]:
schedule=pd.DataFrame('',index=names,columns=shift_id)
for i in names:
    for j in shift_id:
        if x[i,j].x:
            schedule.loc[i,j]= x[i,j].x
schedule.columns=shifts
schedule.head()

day      2019-03-31               2019-04-01               2019-04-02          \
time        Morning Evening Night    Morning Evening Night    Morning Evening   
shift_id         0       1     2          3       4     5          6       7    
name                                                                            
Alexis                                             1                1           
Alyssa            1                                1                        1   
Anthony                         1                                   1           
Brandon                   1                1                                1   
Brianna                   1                1                                    

day            2019-04-03               2019-04-04               2019-04-05  \
time     Night    Morning Evening Night    Morning Evening Night    Morning   
shift_id    8          9       10    11         12      13    14         15   
name                                                                          
Alexis                  1                                1                1   
Alyssa                          1                1                            
Anthony                 1                        1                            
Brandon                                          1                            
Brianna                 1                                                     

day                    2019-04-06                
time     Evening Night    Morning Evening Night  
shift_id      16    17         18      19    20  
name                                             
Alexis                          1                
Alyssa         1                                 
Anthony        1                        1        
Brandon        1                        1        
Brianna

In [24]:
summary=pd.Series(name='Value')
summary['Objective']= mod.objval
summary['Total preference score']= prefsum
summary['Shift inequality'] = U.x-L.x
summary['Night inequality'] = UN.x - LN.x
summary

Objective                -667.0
Total preference score     83.0
Shift inequality            3.0
Night inequality            3.0
Name: Value, dtype: float64

In [25]:
writer=pd.ExcelWriter('small_out.xlsx',datetime_format='m/dd')
schedule.to_excel(writer,sheet_name='Schedule')
summary.to_excel(writer,sheet_name='Summary')
writer.save()

# Data
### need to adjust max # shifts per week constraint, everything else should work fine for big dataset

In [26]:
prefs=pd.read_excel('data.xlsx',header=[0,1,2],sheet_name='Preferences',index_col=0)
reqs =pd.read_excel('data.xlsx',sheet_name='Requirements',index_col=0)
names=prefs.index
shifts=prefs.columns
shift_id=shifts.get_level_values(2)
prefs.columns=shift_id

In [27]:
prefs.head()

shift_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alexis,1,1,1,1,2,1,1,0,2,1,1,2,2,2,1,2,2,2,1,1,2,1,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,0,1,1,1,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,2,1,1,1,1,1,1,2,2,1,1,2,1,1,1,1,2,1,2,1,2,2,2,1,2,0,2,1,1,0,0,2,2,2,2,1,2,1,2,1,1,2,2,1,2,2,0,2,2,2,0,2,1,2,2,0,0,0,0,0,0,0,2,1,1,1,1,2,1,2,2,2,2,1,2,2,2,2,2,1,1,2,1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,2,2,1,1,2,1,0,0,1,2,2,1,1,2,1,1,2,0,2,2
Alyssa,2,2,0,1,2,1,2,2,2,1,2,2,2,1,2,1,2,2,0,0,0,0,0,0,0,0,0,2,1,2,1,1,2,1,1,2,2,2,1,2,1,2,1,1,2,1,1,2,2,0,2,2,2,2,2,2,1,2,1,0,2,2,2,0,1,2,1,1,0,2,1,1,2,2,2,1,2,2,1,1,1,2,2,2,1,2,1,2,2,2,1,1,1,1,2,0,2,2,2,1,2,1,1,2,2,2,1,2,1,0,2,2,1,1,2,1,2,0,0,0,0,0,0,0,0,0,1,2,0,0,2,2,1,1,0,0,0,0,0,1,2,1,2,2,1,2,2,1,2,1,1,1,1,2,2,2,1,1,1,2,2,0,0,1,1,2,2,2,2,1,0,2,2,2,2,2,2,1,1,1,1,1,1,2,0,2,1,2,1
Anthony,1,1,2,1,1,2,2,1,0,2,1,1,2,2,2,1,2,2,1,1,2,2,0,0,0,0,0,0,1,1,2,2,2,2,2,1,2,0,2,2,0,0,2,1,2,2,2,1,2,2,1,1,1,2,2,2,2,2,2,1,0,1,1,1,1,1,2,2,2,2,1,2,1,2,2,2,2,1,2,1,2,2,2,2,2,1,2,2,0,0,0,0,0,2,1,2,1,1,1,2,2,1,2,2,1,1,2,2,1,2,2,1,2,1,1,2,1,1,0,0,0,0,0,0,1,2,2,1,2,1,2,2,2,2,2,2,1,2,1,2,1,1,2,1,2,1,2,2,2,1,2,2,2,2,1,2,2,1,1,1,2,2,2,2,1,2,1,2,0,2,1,1,2,2,2,2,1,2,2,2,2,2,1,1,1,1,2,1,1
Brandon,1,1,2,2,2,1,1,2,1,0,0,0,2,1,0,0,1,2,2,2,1,2,1,1,2,1,2,0,1,2,2,1,2,1,0,0,0,1,1,2,2,2,1,1,2,0,2,0,1,2,2,1,2,1,2,2,1,2,1,2,1,2,2,1,2,2,1,1,0,2,2,2,2,2,1,1,2,1,1,1,0,0,0,0,2,2,1,1,2,1,2,1,1,0,0,0,0,0,2,1,1,1,2,1,2,2,1,2,1,1,2,2,1,2,2,1,1,1,2,1,2,0,0,0,1,1,2,1,1,0,0,0,0,0,0,0,0,2,1,2,1,1,2,2,1,1,2,2,0,0,0,2,2,2,2,1,1,2,1,1,2,1,1,1,1,1,2,1,2,1,1,2,1,1,0,1,2,0,2,1,2,1,1,1,2,1,1,1,1
Brianna,0,2,1,1,1,2,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,2,1,1,1,2,1,2,1,2,0,1,1,2,1,1,1,1,1,0,2,2,1,0,0,0,0,0,0,0,0,0,0,1,1,2,2,2,1,1,1,2,2,1,2,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,2,1,0,2,2,1,1,2,2,2,2,1,2,2,2,1,1,1,1,2,1,1,1,2,1,2,1,2,2,2,1,1,2,2,1,1,2,2,2,1,0,2,2,2,1,1,1,2,2,1,1,2,1,1,1,2,2,2,1,0,1,1,2,1,2,2,1,1,1,2,1,0,1,1,2,1,2,2,1,2,2,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [28]:
I = names           #set of nurses
J = shift_id        #set of shifts
n = int(len(J)/21)  #number of weeks
W = range(n)        #set of weeks
d = reqs['persons'] #required # of nurses for each shift
nights = J[2::3]    #shift id of every night shift
z = []              #binary auxillary variable for whether or not it is a night shift
for j in J:
    if j in nights:
        z.append(1)
    else:
        z.append(0)
weeklist = []       #creating list where each element represents a week, each element has 21 shifts in it
a=0
for i in W:
    weeklist.append(J[a:a+21])
    a+=21
#prefs.loc[i,j]     #nurse i's preference for shift j

In [33]:
weeklist[8]

Int64Index([168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
            181, 182, 183, 184, 185, 186, 187, 188],
           dtype='int64', name='shift_id')

In [53]:
mod = Model()

x = mod.addVars(I,J,vtype=GRB.BINARY)        #DV: whether nurse i works shift j, binary
total = mod.addVars(I,vtype=GRB.INTEGER)     #total # of shifts worked for each nurse
totaln=mod.addVars(I,vtype=GRB.INTEGER)      #total # of night shift worked for each nurse
U = mod.addVar(vtype=GRB.INTEGER)            #max # of shifts worked across all nurses
L = mod.addVar(vtype=GRB.INTEGER)            #min # of shifts worked across all nurses
UN = mod.addVar(vtype=GRB.INTEGER)           #max # of night shifts worked across all nurses
LN = mod.addVar(vtype=GRB.INTEGER)           #min # of night shifts worked across all nurses

mod.setObjective(sum(x[i,j]*prefs.loc[i,j] for i in I for j in J) - 100*(U-L) - 150*(UN-LN), sense=GRB.MAXIMIZE)

mod.addConstr(U==max_([total[i] for i in I]))   #max total shifts
mod.addConstr(L==min_([total[i] for i in I]))   #min total shifts
mod.addConstr(UN==max_([totaln[i] for i in I])) #max total night shifts
mod.addConstr(LN==min_([totaln[i] for i in I])) #min total night shifts

for i in I:                                     #total number of shifts and nightshifts scheduled for each nurse
    mod.addConstr(total[i]==sum(x[i,j] for j in J))
for i in I:
    mod.addConstr(totaln[i]==sum(x[i,j]*z[j]for j in J))

for j in J:                                     #shift demand
    mod.addConstr(sum(x[i,j] for i in I)==d[j])
    
for i in I:                                     #less than 6 shifts a week
    for w in W:
        mod.addConstr(sum(x[i,j] for j in weeklist[w])<=6)
    
for i in I:                                     #no consecutive shifts
    for j in J[1:]:
        mod.addConstr(x[i,j-1]+x[i,j]<=1)
for i in I:
    for j in J[:-1]:
        mod.addConstr(x[i,j+1]+x[i,j]<=1)
        
for i in I:                                     #night shift constraints
    for j in J[2:]:
        mod.addConstr(x[i,j-2]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[1:]:
        mod.addConstr(x[i,j-1]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[:-1]:
        mod.addConstr(x[i,j+1]<=1-(z[j]*x[i,j]))
for i in I: 
    for j in J[:-2]:
        mod.addConstr(x[i,j+2]<=1-(z[j]*x[i,j]))

for i in I:                                     #blackout shift
    for j in J:
        mod.addConstr(x[i,j]<=prefs.loc[i,j])

mod.setParam('outputflag',False)
mod.optimize()
mod.objval
prefsum = (sum(x[i,j].x*prefs.loc[i,j] for i in I for j in J)) #sum of preference score
print('Objective value: ', mod.objval)
print('Sum of preference score: ',prefsum)
print('Shift inequality: ', U.x-L.x)   #shift inequality
print('Night shift inequality: ', UN.x-LN.x) #night shift inequality

Objective value:  3633.0
Sum of preference score:  4183.0
Shift inequality:  4.0
Night shift inequality:  1.0


In [52]:
for j in J:
    print(sum(x[i,j].x for i in I),d[j])
    

18.0 15
16.0 15
6.0 5
15.0 15
15.0 15
5.0 5
17.0 15
15.0 15
5.0 5
15.0 15
15.0 15
6.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
16.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
6.0 5
15.0 15
15.0 15
6.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
6.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
18.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
16.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
6.0 5
16.0 15
15.0 15
8.0 5
15.0 15
15.0 15
8.0 5
15.0 15
15.0 15
6.0 5
16.0 15
17.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
16.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15.0 15
5.0 5
15.0 15
15

In [54]:
schedule=pd.DataFrame('',index=names,columns=shift_id)
for i in names:
    for j in shift_id:
        if x[i,j].x:
            schedule.loc[i,j]= x[i,j].x
schedule.columns=shifts
schedule.head()

day      2019-03-31               2019-04-01               2019-04-02          \
time        Morning Evening Night    Morning Evening Night    Morning Evening   
shift_id        0       1     2          3       4     5          6       7     
name                                                                            
Alexis                                             1                            
Alyssa                    1                        1                        1   
Anthony                         1                                   1           
Brandon                                    1                                1   
Brianna                   1                              1                      

day            2019-04-03               2019-04-04               2019-04-05  \
time     Night    Morning Evening Night    Morning Evening Night    Morning   
shift_id   8          9       10    11         12      13    14         15    
name                                                                          
Alexis       1                                   1                            
Alyssa                          1                1                            
Anthony                 1                        1                            
Brandon                                          1                            
Brianna                 1                                                     

day                    2019-04-06               2019-04-07                \
time     Evening Night    Morning Evening Night    Morning Evening Night   
shift_id     16    17         18      19    20         21      22    23    
name                                                                       
Alexis         1                              1                            
Alyssa         1                                                           
Anthony        1                                         1                 
Brandon                                 1                1                 
Brianna                                                          1         

day      2019-04-08               2019-04-09               2019-04-10          \
time        Morning Evening Night    Morning Evening Night    Morning Evening   
shift_id        24      25    26         27      28    29         30      31    
name                                                                            
Alexis            1                                1                        1   
Alyssa                                     1                                    
Anthony                                                             1           
Brandon                         1                        1                      
Brianna                   1                                                 1   

day            2019-04-11               2019-04-12               2019-04-13  \
time     Night    Morning Evening Night    Morning Evening Night    Morning   
shift_id   32         33      34    35         36      37    38         39    
name                                                                          
Alexis                          1                1                            
Alyssa       1                        1                        1              
Anthony                         1                1                        1   
Brandon      1                                                                
Brianna                 1                                1                    

day                    2019-04-14               2019-04-15                \
time     Evening Night    Morning Evening Night    Morning Evening Night   
shift_id     40    41         42      43    44         45      46    47    
name                                                                       
Alexis         1                        1                        1         
Alyssa               1                        1    

In [55]:
summary=pd.Series(name='Value')
summary['Objective']= mod.objval
summary['Total preference score']= prefsum
summary['Shift inequality'] = U.x-L.x
summary['Night inequality'] = UN.x - LN.x
summary

Objective                 3633.0
Total preference score    4183.0
Shift inequality             4.0
Night inequality             1.0
Name: Value, dtype: float64

In [56]:
writer=pd.ExcelWriter('test_out.xlsx',datetime_format='m/dd')
schedule.to_excel(writer,sheet_name='Schedule')
summary.to_excel(writer,sheet_name='Summary')
writer.save()